# NLP and Neural Networks

Francisco Roh

Bryan Calisto

Este ejercicio fue realizado en Google Colab.


In this exercise, we'll apply our knowledge of neural networks to process natural language. As we did in the bigram exercise, the goal of this lab is to predict the next word, given the previous one.

### Data set

Load the text from "One Hundred Years of Solitude" that we used in our bigrams exercise. It's located in the data folder.

### Important note:

Start with a smaller part of the text. Maybe the first 10 parragraphs, as the number of tokens rapidly increases as we add more text.

Later you can use a bigger corpus.

In [374]:
from nltk.tokenize import TreebankWordTokenizer

In [375]:
text = open('./cap1.txt', 'r').read().lower()

In [376]:
text

'muchos años después, frente al pelotón de fusilamiento, el coronel aureliano buendía había de recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. macondo era entonces una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos prehistóricos. el mundo era tan reciente, que muchas cosas carecían de nombre, y para mencionarlas había que señalarlas con el dedo. todos los años, por el mes de marzo, una familia de gitanos desarrapados plantaba su carpa cerca de la aldea, y con un grande alboroto de pitos y timbales daban a conocer los nuevos inventos. primero llevaron el imán. un gitano corpulento, de barba montaraz y manos de gorrión, que se presentó con el nombre de melquíades, hizo una truculenta demostración pública de lo que él mismo llamaba la octava maravilla de los sabios alquimistas de macedonia. fue de casa en casa arrastrando dos lingote

Don't forget to prepare the data by generating the corresponding tokens.

In [377]:
# Paso 1: Crear el tokenizador
tokenizer = TreebankWordTokenizer()

In [378]:
# Paso 2: Tokenizar el texto
tokens = tokenizer.tokenize(text)

In [379]:
tokens[:50]

['muchos',
 'años',
 'después',
 ',',
 'frente',
 'al',
 'pelotón',
 'de',
 'fusilamiento',
 ',',
 'el',
 'coronel',
 'aureliano',
 'buendía',
 'había',
 'de',
 'recordar',
 'aquella',
 'tarde',
 'remota',
 'en',
 'que',
 'su',
 'padre',
 'lo',
 'llevó',
 'a',
 'conocer',
 'el',
 'hielo.',
 'macondo',
 'era',
 'entonces',
 'una',
 'aldea',
 'de',
 'veinte',
 'casas',
 'de',
 'barro',
 'y',
 'cañabrava',
 'construidas',
 'a',
 'la',
 'orilla',
 'de',
 'un',
 'río',
 'de']

In [380]:
len(tokens)

6293

In [381]:
min(len(x) for x in tokens)

1

In [382]:
max(len(x) for x in tokens)

16

### Let's prepare the data set.

Our neural network needs to have an input X and an output y. Remember that these sets are numerical, so you'd need something to map the tokens into numbers, and viceversa.

In [383]:
# in this case, let's consider a bigram (w1, w2)
# assign the w1 to the X vector, and w2 to the y vector, why do we do this?

Al asignar w1 a X y w2 a y, estamos enseñando a la red neuronal a aprender la transición de una palabra a la siguiente.

Este enfoque permite a la red neuronal aprender patrones en el lenguaje, como la probabilidad de que ciertas palabras sigan a otras. Por ejemplo, si w1 es "hola", la red aprenderá que "mundo" es una posible w2, si es una frase muy utilizada.

Este método es una forma de entrenamiento supervisado donde X es la entrada que se alimenta al modelo y y es la etiqueta o salida que queremos que el modelo prediga. En este caso, y es la palabra que sigue a X.

In [384]:
# Don't forget that since we are using torch, our training set vectors should be tensors

In [385]:
import torch
from collections import Counter

# Paso 2: Construir el vocabulario manualmente
token_freq = Counter(tokens)  # Contar la frecuencia de cada token
vocab = {word: i+1 for i, (word, _) in enumerate(token_freq.items())}  # Crear el vocabulario (i+1 para que los índices comiencen en 1)
vocab["<unk>"] = 0  # Añadir un token desconocido

# Paso 3: Convertir tokens en índices usando el vocabulario
sequence = [vocab.get(token, vocab["<unk>"]) for token in tokens]

In [386]:
# Paso 4: Crear bigramas (X, y) para el entrenamiento
X = []
y = []

for i in range(len(sequence) - 1):
    X.append(sequence[i])     # w1
    y.append(sequence[i + 1]) # w2

# Convertir a tensores de PyTorch
X = torch.tensor(X, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)

# Verificar
print("X:", X[:20])  # Mostrar las primeras 10 entradas de X
print("y:", y[:20])  # Mostrar las primeras 10 salidas correspondientes

X: tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9,  4, 10, 11, 12, 13, 14,  8, 15, 16,
        17, 18])
y: tensor([ 2,  3,  4,  5,  6,  7,  8,  9,  4, 10, 11, 12, 13, 14,  8, 15, 16, 17,
        18, 19])


In [387]:
# Note that our vectors are integers, which can be thought as a categorical variables.
# torch provides the one_hot method, that would generate tensors suitable for our nn
# make sure that the dtype of your tensor is float.

In [388]:
# Paso 5: Convertir X a una representación one-hot
X_one_hot = torch.nn.functional.one_hot(X, num_classes=len(vocab)).float()

# Verificar
print("X_one_hot:", X_one_hot[:10])  # Mostrar las primeras 10 entradas de X one-hot
print("y:", y[:10])  # Mostrar las primeras 10 salidas correspondientes
print("X_one_hot shape:", X_one_hot.shape)


X_one_hot: tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
y: tensor([ 2,  3,  4,  5,  6,  7,  8,  9,  4, 10])
X_one_hot shape: torch.Size([6292, 2128])


### Network design
To start, we are going to have a very simple network. Define a single layer network

In [389]:
# How many neurons should our input layer have?
# Use as many neurons as the total number of categories (from your one-hot encoded tensors)

Para definir una red neuronal simple de una sola capa, la cantidad de neuronas en la capa de entrada debe coincidir con el número total de categorías (es decir, el tamaño del vocabulario). Esto se debe a que cada neurona en la capa de entrada recibe una parte de la información codificada en el tensor one-hot, donde cada categoría corresponde a una neurona específica.

In [390]:
# Use the softmax as your activation layer

In [391]:
# Train your network

In [392]:
!pip install pytorch-lightning

In [393]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim

# Definir el modelo con PyTorch Lightning
class SingleLayerNN(pl.LightningModule):
    def __init__(self, input_size, output_size):
        super(SingleLayerNN, self).__init__()
        # Capa lineal
        self.linear = nn.Linear(input_size, output_size)
        # Capa de activación softmax
        self.softmax = nn.Softmax(dim=1)
        # Definir la función de pérdida como NLLLoss()
        self.criterion = nn.NLLLoss()

    def forward(self, x):
        # Aplicar la capa lineal y luego la capa softmax
        out = self.linear(x)
        out = self.softmax(out)
        return out

    def training_step(self, batch, batch_idx):
        batch_X, batch_y = batch
        outputs = self.forward(batch_X)
        log_outputs = torch.log(outputs)  # Aplica torch.log para NLLLoss
        loss = self.criterion(log_outputs, batch_y)  # Usa NLLLoss
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)  # Aquí se configura para la barra de progreso
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [394]:
from torch.utils.data import TensorDataset, DataLoader

# Definir el DataModule para la gestión de datos
class TextoDataModule(pl.LightningDataModule):
    def __init__(self, X, y, batch_size=32):
        super().__init__()
        self.X = X
        self.y = y
        self.batch_size = batch_size

    def setup(self, stage=None):
        # Crear un dataset
        self.dataset = TensorDataset(self.X, self.y)

    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)


In [395]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.progress import RichProgressBar

PATIENCE = 5

# Crear una instancia del modelo
input_size = len(vocab)
output_size = len(vocab)
model = SingleLayerNN(input_size=input_size, output_size=output_size)

# Crear un DataModule con los datos de entrenamiento
data_module = TextoDataModule(X_one_hot, y, batch_size=32)

# Configurar el trainer de PyTorch Lightning
trainer = pl.Trainer(max_epochs=10)

#callback_tqdm = RichProgressBar(leave=True)
callback_early = EarlyStopping(monitor="train_loss", mode="min", patience=PATIENCE)

trainer = pl.Trainer(
    callbacks=[callback_early],
    max_epochs=20,
    accelerator="auto",  # Uses GPUs or TPUs if available
    devices="auto",  # Uses all available GPUs/TPUs if applicable
    )

# Entrenar el modelo
trainer.fit(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | linear    | Linear  | 4.5 M  | train
1 | softmax   | Softmax | 0      | train
2 | criterion | NLLLoss | 0      | train
----------------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
18.122    Total estimated model params s

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


### Analysis

**1. Test your network with a few words**

In [396]:
# Función para convertir una palabra a su representación one-hot
def word_to_one_hot(word, vocab):
    word_idx = vocab.get(word, vocab["<unk>"])
    one_hot_vector = torch.nn.functional.one_hot(torch.tensor(word_idx), num_classes=len(vocab)).float()
    return one_hot_vector.unsqueeze(0)  # Añadir una dimensión para batch

# Función para convertir el índice predicho a la palabra correspondiente en el vocabulario
def idx_to_word(index, vocab):
    word_list = list(vocab.keys())
    idx_list = list(vocab.values())
    if index in idx_list:
        return word_list[idx_list.index(index)]
    else:
        return "<unk>"  # Devuelve un token desconocido si el índice no está en el vocabulario

# Palabras de prueba del vocabulario
test_words = [tokens[0], 'Aureliano', tokens[50], tokens[100], tokens[150], tokens[200], tokens[250]]  # Seleccionar palabras de la lista tokens

# Probar cada palabra
model.eval()  # Asegura que el modelo esté en modo de evaluación
with torch.no_grad():  # Desactivar el cálculo de gradientes para las pruebas
    for word in test_words:
        one_hot = word_to_one_hot(word, vocab)
        output = model(one_hot)  # Pasar la representación one-hot por la red

        # Obtener el índice de la palabra predicha
        predicted_idx = torch.argmax(output, dim=1).item()
        predicted_word = idx_to_word(predicted_idx, vocab)

        print(f"Palabra de entrada: {word}")
        print(f"Palabra predicha: {predicted_word}")
        print(f"Salida: {output}")
        print("-" * 50)


Palabra de entrada: muchos
Palabra predicha: ,
Salida: tensor([[5.6844e-05, 4.6490e-04, 2.8248e-03,  ..., 3.9214e-04, 3.6145e-04,
         3.8275e-04]])
--------------------------------------------------
Palabra de entrada: Aureliano
Palabra predicha: ,
Salida: tensor([[4.8905e-05, 4.6911e-04, 1.0983e-03,  ..., 3.9173e-04, 3.5238e-04,
         3.6583e-04]])
--------------------------------------------------
Palabra de entrada: aguas
Palabra predicha: de
Salida: tensor([[5.3406e-05, 4.6463e-04, 1.0773e-03,  ..., 3.9337e-04, 3.5801e-04,
         3.7823e-04]])
--------------------------------------------------
Palabra de entrada: una
Palabra predicha: aldea
Salida: tensor([[7.1628e-05, 3.9461e-04, 7.9107e-04,  ..., 3.4740e-04, 3.1968e-04,
         3.2561e-04]])
--------------------------------------------------
Palabra de entrada: de
Palabra predicha: la
Salida: tensor([[2.9202e-05, 1.3018e-04, 2.3676e-04,  ..., 1.1602e-04, 1.5749e-03,
         1.1046e-04]])
------------------------------

**2. What does each value in the tensor represents?**

  Cada valor en el tensor de salida representa la probabilidad de que una palabra específica en el vocabulario sea la siguiente palabra en la secuencia, según el modelo. Después de aplicar la función softmax, los valores en el tensor suman 1, y cada valor corresponde a la probabilidad de que cada palabra en el vocabulario sea la palabra correcta que sigue.

**3. Why does it make sense to choose that number of neurons in our layer?**

  El número de neuronas en la capa de salida es igual al tamaño del vocabulario. Esto tiene sentido porque la tarea es predecir la siguiente palabra en la secuencia, que podría ser cualquier palabra del vocabulario. Cada neurona en la capa de salida corresponde a una palabra del vocabulario, y la salida de esa neurona representa la confianza del modelo en que esa palabra es la correcta.

**4. What's the negative likelihood for each example?**

  La probabilidad logarítmica negativa (NLL, por sus siglas en inglés) para cada ejemplo es una medida de cuán probable es la etiqueta verdadera (la palabra correcta que sigue en la secuencia) según la distribución de probabilidad predicha por el modelo.

  Matemáticamente, si el modelo asigna una probabilidad p(yi|xi) a la etiqueta verdadera yi dada la entrada xi, la NLL se calcula como:

    NLL = - log(p(yi|xi))

  En la práctica, la función **CrossEntropyLoss** en PyTorch combina la activación softmax y el cálculo de la NLL, por lo que cuando se entrena el modelo se está minimizando la probabilidad logarítmica negativa. Si usamos como capa de activación **Softmax**, se puede usar como medida **NLLLoss**, y la pérdida se obtiene calculando previamente el logaritmo de los valores de salida de la softmax.

**5. Try generating a few sentences?**


Creamos la función para generar texto,  permitiendo seleccionar entre diferentes métodos de generación, como seleccionar la palabra con la mayor probabilidad, o utilizar otros métodos como sampling (muestreo) o top-k sampling:

In [397]:
import torch

def generar_oracion(palabra_inicial, modelo, vocabulario, longitud_maxima=10, metodo="maximo", top_k=5):
    modelo.eval()  # Configurar el modelo en modo de evaluación
    palabra_actual = palabra_inicial
    oracion = [palabra_actual]

    # Configurar el generador para muestreo con una semilla fija
    g = torch.Generator()
    g.manual_seed(47)

    with torch.no_grad():
        for _ in range(longitud_maxima - 1):
            # Convertir la palabra actual a su representación one-hot
            one_hot = word_to_one_hot(palabra_actual, vocabulario)
            # Obtener la predicción del modelo
            salida = modelo(one_hot)

            if metodo == "maximo":
                # Seleccionar la palabra con la mayor probabilidad
                siguiente_palabra_idx = torch.argmax(salida, dim=1).item()

            elif metodo == "sampling":
                # Seleccionar una palabra mediante muestreo probabilístico
                siguiente_palabra_idx = torch.multinomial(salida, num_samples=1, generator=g).item()

            elif metodo == "top-k":
                # Seleccionar una palabra dentro de las top-k palabras más probables
                top_k_indices = torch.topk(salida.squeeze(), top_k).indices
                siguiente_palabra_idx = torch.multinomial(salida[0, top_k_indices], num_samples=1, generator=g).item()
                siguiente_palabra_idx = top_k_indices[siguiente_palabra_idx].item()

            else:
                raise ValueError("Método no reconocido. Use 'maximo', 'sampling' o 'top-k'.")

            # Obtener la palabra correspondiente al índice
            siguiente_palabra = list(vocabulario.keys())[list(vocabulario.values()).index(siguiente_palabra_idx)]
            oracion.append(siguiente_palabra)
            palabra_actual = siguiente_palabra

    return ' '.join(oracion)


In [398]:
palabra_inicial = tokens[0]
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="maximo")
print(f"Oración generada con: {palabra_inicial}, metodo máximo: {oracion_generada}\n")

palabra_inicial = tokens[0]
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="sampling")
print(f"Oración generada con: {palabra_inicial}, metodo sampling: {oracion_generada}\n")

palabra_inicial = tokens[0]
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="top-k")
print(f"Oración generada con: {palabra_inicial}, metodo top-k: {oracion_generada}\n")

palabra_inicial = "Aureliano"
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="maximo")
print(f"Oración generada con: {palabra_inicial}, metodo máximo: {oracion_generada}\n")

palabra_inicial = "Aureliano"
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="sampling")
print(f"Oración generada con: {palabra_inicial}, metodo sampling: {oracion_generada}\n")

palabra_inicial = "Aureliano"
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="top-k")
print(f"Oración generada con: {palabra_inicial}, método top-k: {oracion_generada}\n")

Oración generada con: muchos, metodo máximo: muchos , y de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de

Oración generada con: muchos, metodo sampling: muchos conjuro engendro exhalaba profusión atanor mordiente confirmaron ñame recuerdo arcadio los día cocina en demonio cálculos trató chivos limpios público calle tocarlo» suelo. castaño ver oro recuerdo lupa. coronel estructura colocaba negó caramelo reciente lupa. abuelo— magnífico. , y el condición muros voluntarioso «lo sendero se milagro invención voluntad

Oración generada con: muchos, metodo top-k: muchos y la de la de la de su casa , la de la de la aldea de la de su vida de que , de la de la de los hombres y de la mano , de que en que el gitano y de los años en que de

Oración generada con: Aureliano, metodo máximo: Aureliano , y de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la de la 

**Cálculo de NLL:**

In [399]:
import torch

# Suponiendo que 'X_one_hot' es el tensor de entrada (las representaciones one-hot) y 'y' son las etiquetas verdaderas
model.eval()  # Asegurarse de que el modelo esté en modo de evaluación
with torch.no_grad():  # Desactivar el cálculo de gradientes para la evaluación
    outputs = model(X_one_hot)  # Pasar todo el conjunto de entradas por el modelo

# Calcular la NLL para cada ejemplo
nll_per_example = -torch.log(torch.gather(torch.softmax(outputs, dim=1), 1, y.unsqueeze(1)))

# Convertir a un formato más legible y mostrar los resultados
nll_per_example = nll_per_example.squeeze()
for i, nll in enumerate(nll_per_example):
    print(f"Ejemplo {i + 1}: NLL = {nll.item()}")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Ejemplo 1293: NLL = 7.661925315856934
Ejemplo 1294: NLL = 7.662755966186523
Ejemplo 1295: NLL = 7.652911186218262
Ejemplo 1296: NLL = 7.635180473327637
Ejemplo 1297: NLL = 7.6590800285339355
Ejemplo 1298: NLL = 7.662611961364746
Ejemplo 1299: NLL = 7.657351493835449
Ejemplo 1300: NLL = 7.660799026489258
Ejemplo 1301: NLL = 7.636510848999023
Ejemplo 1302: NLL = 7.644414901733398
Ejemplo 1303: NLL = 7.660689830780029
Ejemplo 1304: NLL = 7.63838529586792
Ejemplo 1305: NLL = 7.662003040313721
Ejemplo 1306: NLL = 7.638640880584717
Ejemplo 1307: NLL = 7.636852741241455
Ejemplo 1308: NLL = 7.54460334777832
Ejemplo 1309: NLL = 7.659437656402588
Ejemplo 1310: NLL = 7.656853675842285
Ejemplo 1311: NLL = 7.6608734130859375
Ejemplo 1312: NLL = 7.638721466064453
Ejemplo 1313: NLL = 7.662073612213135
Ejemplo 1314: NLL = 7.6382927894592285
Ejemplo 1315: NLL = 7.629778861999512
Ejemplo 1316: NLL = 7.658648490905762
Ejemplo 1317: NLL = 

6. What's the negative likelihood for each sentence?

  La probabilidad logarítmica negativa para una oración completa sería la suma de las probabilidades logarítmicas negativas de cada par de palabras en la oración. Es decir, si generas una oración de longitud
  n, la NLL total sería:

  


$$
\text{NLL total} = - \sum_{i=1}^{n-1} \log(p(y_{i+1} \mid y_i))
$$

  Cada p(yi+1|yi) es la probabilidad de que la palabra yi+1 siga a yi, según lo predicho por el modelo

In [400]:
def calcular_nll_oracion(oracion, modelo, vocabulario):
    modelo.eval()  # Configurar el modelo en modo de evaluación
    nll_total = 0.0
    palabra_actual = oracion[0]

    with torch.no_grad():  # Desactivar el cálculo de gradientes para la evaluación
        for siguiente_palabra in oracion[1:]:
            # Convertir la palabra actual en su representación one-hot
            one_hot = word_to_one_hot(palabra_actual, vocabulario)
            salida = modelo(one_hot)

            # Obtener el índice de la siguiente palabra
            siguiente_palabra_idx = vocabulario.get(siguiente_palabra, vocabulario["<unk>"])

            # Calcular la NLL para esta transición
            nll = -torch.log(torch.softmax(salida, dim=1)[0, siguiente_palabra_idx])
            nll_total += nll.item()

            # Pasar a la siguiente palabra
            palabra_actual = siguiente_palabra

    return nll_total

# Ejemplo de uso
oracion = [tokens[0], tokens[1], tokens[2], tokens[3], tokens[4], tokens[5],tokens[6], tokens[7], tokens[8]]
nll_oracion = calcular_nll_oracion(oracion, model, vocab)
print(f"NLL para la oración: {nll_oracion}")


NLL para la oración: 61.22891092300415


### Design your own neural network (more layers and different number of neurons)
The goal is to get sentences that make more sense

In [401]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.progress import RichProgressBar

PATIENCE = 5

class MultiLayerNN(pl.LightningModule):
    def __init__(self, input_size, output_size):
        super(MultiLayerNN, self).__init__()
        # Definir una red neuronal más compleja
        self.layer1 = nn.Linear(input_size, 32)
        self.layer2 = nn.Linear(32, 16)
        #self.layer3 = nn.Linear(16, 16)
        self.output_layer = nn.Linear(16, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.criterion = nn.NLLLoss()

    def forward(self, x):
        # Paso hacia adelante a través de las capas
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        #x = self.relu(self.layer3(x))
        x = self.output_layer(x)
        out = self.softmax(x)
        return out

    def training_step(self, batch, batch_idx):
        batch_X, batch_y = batch
        outputs = self.forward(batch_X)
        log_outputs = torch.log(outputs)  # Aplica torch.log para NLLLoss
        loss = self.criterion(log_outputs, batch_y)  # Usa NLLLoss
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)  # Aquí se configura para la barra de progreso
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Crear una instancia del modelo
input_size = len(vocab)
output_size = len(vocab)
model = MultiLayerNN(input_size=input_size, output_size=output_size)

# Crear un DataModule con los datos de entrenamiento
data_module = TextoDataModule(X_one_hot, y, batch_size=32)

#callback_tqdm = RichProgressBar(leave=True)
callback_early = EarlyStopping(monitor="train_loss", mode="min", patience=PATIENCE)

# Configurar el entrenador
trainer = pl.Trainer(
    callbacks=[callback_early],
    max_epochs=20,
    accelerator="auto",  # Usa GPUs o TPUs si están disponibles
    devices="auto",  # Usa todas las GPUs/TPUs disponibles si es aplicable
)

# Entrenar el modelo
trainer.fit(model, data_module)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type    | Params | Mode 
-------------------------------------------------
0 | layer1       | Linear  | 68.1 K | train
1 | layer2       | Linear  | 528    | train
2 | output_layer | Linear  | 36.2 K | train
3 | relu         | ReLU    | 0      | train
4 | softmax      | Softmax | 0      | train
5 | criterion    | NLLLoss | 0      | train
-------------------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.419     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [402]:
palabra_inicial = tokens[0]
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="maximo")
print(f"Oración generada con: {palabra_inicial}, metodo máximo: {oracion_generada}\n")

palabra_inicial = tokens[0]
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="sampling")
print(f"Oración generada con: {palabra_inicial}, metodo sampling: {oracion_generada}\n")

palabra_inicial = tokens[0]
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="top-k")
print(f"Oración generada con: {palabra_inicial}, metodo top-k: {oracion_generada}\n")

palabra_inicial = "Aureliano"
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="maximo")
print(f"Oración generada con: {palabra_inicial}, metodo máximo: {oracion_generada}\n")

palabra_inicial = "Aureliano"
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="sampling")
print(f"Oración generada con: {palabra_inicial}, metodo sampling: {oracion_generada}\n")

palabra_inicial = "Aureliano"
oracion_generada = generar_oracion(palabra_inicial, model, vocab, longitud_maxima=50, metodo="top-k")
print(f"Oración generada con: {palabra_inicial}, método top-k: {oracion_generada}\n")

Oración generada con: muchos, metodo máximo: muchos , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un

Oración generada con: muchos, metodo sampling: muchos aquella engendro de la atanor mordiente confirmaron que no arcadio buendía y cocina en los cálculos trató de un explicación febril se decir , puso con los olvidó de toda colocaba del caramelo esperaban una abuelo— magnífico. , y el dinero muros de en la se pero todas voluntad

Oración generada con: muchos, metodo top-k: muchos en que los y el los niños se al vez del río , y el mundo se , y de su vez de su aldea , y el los niños no con la aldea del río y un sierra en el mundo a la mano para la padre se

Oración generada con: Aureliano, metodo máximo: Aureliano , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un aldea , y el un

Oración generada c

Se mantiene el problema de que las oraciones generadas son repetitivas y poco coherentes (siendo con sampling la mejor generación), posiblemente a que se trata de un modelo simple con un conjunto de datos pequeño que tiene dificultades para capturar la estructura del lenguaje y el contexto respectivo ya que solo usamos una palabra para predecir la siguiente.

Al usar más capas o más neuronas, los resultados son peores y repetitivos, aparentando un sobreajuste de las probabilidades que encuentra, llevando inclusive a registrar valores extremos que generan posteriormente errores.